In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
       # print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai import *
from fastai.vision import *

In [ ]:
!pip install faiss
!pip install faiss-gpu
import faiss
#import faiss-gpu

In [ ]:
!git clone https://github.com/amarmrf/google-retrieval-challenge-2019-fastai-starter

In [ ]:
#https://github.com/filipradenovic/cnnimageretrieval-pytorch/blob/master/cirtorch/utils/download.py
#from third_party import download

import sys
sys.path.insert(1, 'google-retrieval-challenge-2019-fastai-starter/third_party')
import download

In [ ]:
DATA_DIR='data/'
!mkdir $DATA_DIR

In [ ]:
download.download_test(DATA_DIR)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
plt.imshow(Image.open('data/test/oxford5k/jpg/oxford_001195.jpg'))

In [ ]:
download.download_train(DATA_DIR)
#takes ~40 min

In [ ]:
#from third_party.datahelpers import cid2filename
from datahelpers import cid2filename
import pickle

with open('data/train/retrieval-SfM-120k/retrieval-SfM-120k.pkl', 'rb') as f:
    db = pickle.load(f)['train']
    
images = [cid2filename(db['cids'][i], 'data/train/retrieval-SfM-120k/ims/') for i in range(len(db['cids']))]
clusters = db['cluster']
qpool = db['qidxs']
ppool = db['pidxs']
qpidxs = list(set([(qpool[i], ppool[i]) for i in range(len(ppool))]))

In [ ]:
#Gather all "query+positive pool" into classes
invert_list = {}
subclusters = {}
subidx = 0
for q,p in qpidxs:
    #print (q,p)
    if q not in invert_list and p not in invert_list:
        subclusters[subidx] = []
        subclusters[subidx].append(p)
        subclusters[subidx].append(q)
        invert_list[p] = subidx
        invert_list[q] = subidx
        subidx+=1
    else:
        if p in invert_list:
            exist_sub_idx = invert_list[p]
            invert_list[q] = exist_sub_idx
        else:
            exist_sub_idx = invert_list[q]
            invert_list[p] = exist_sub_idx
        subclusters[exist_sub_idx].append(p)
        subclusters[exist_sub_idx].append(q)
print (len(subclusters))

In [ ]:
#Here we create full csv and small (for quick experimenting)
import os
count_no_class= 0
prefix = os.path.join(DATA_DIR, 'train/retrieval-SfM-120k/ims/')

with open('train_sfm120k_1000_classes.csv', 'w') as f:
    f.write('Image,cluster,Id\n')
    for i in range(len(images)):
        img = images[i].replace(prefix,'')
        if i in invert_list:
            if invert_list[i] > 1000:
                continue
            f.write(img+','+str(clusters[i])+','+str(invert_list[i])+'\n')
print (count_no_class, len(images))
with open('train_sfm120k_full.csv', 'w') as f:
    f.write('Image,cluster,Id\n')
    for i in range(len(images)):
        img = images[i].replace(prefix,'')
        if i in invert_list:
            f.write(img+','+str(clusters[i])+','+str(invert_list[i])+'\n')
        else:
            f.write(img+','+str(clusters[i])+',-1\n')
            count_no_class+=1
print (count_no_class, len(images))

# Val no train

In [ ]:
#import sys
#sys.path.insert(1, 'google-retrieval-challenge-2019-fastai-starter/third_party')
import matplotlib.pyplot as plt
from testdataset import configdataset
from evaluate import compute_map_and_print

from fastai import *
from fastai.vision import *
import pandas as pd

In [ ]:
dataset ='oxford5k'
DATA_DIR='data/'
cfg = configdataset(dataset, os.path.join(DATA_DIR, 'test'))
images = [cfg['im_fname'](cfg,i) for i in range(cfg['n'])]
qimages = [cfg['qim_fname'](cfg,i) for i in range(cfg['nq'])]

In [ ]:
df = pd.DataFrame(images, columns=['Image'])
qdf  = pd.DataFrame(qimages, columns=['qimages'])
bbxs = {qimages[i]:tuple(cfg['gnd'][i]['bbx']) for i in range(cfg['nq'])}

In [ ]:
BS=1
NUM_WORKERS=8
tfms = get_transforms(do_flip=False)
tfms = (tfms[1],tfms[1]) #no transforms
data = (ImageList.from_df(df,'', cols=['Image'])
        .split_none()
        .label_const()
        .transform(tfms, resize_method=ResizeMethod.NO)
        .databunch(bs=BS, num_workers=NUM_WORKERS)
        .normalize(imagenet_stats)
       ) 
data.train_dl.dl.batch_sampler.sampler = torch.utils.data.SequentialSampler(data.train_ds)
data.train_dl.dl.batch_sampler.drop_last = False
#No shuffle
data.show_batch()


In [ ]:
import sys
sys.path.insert(1, 'google-retrieval-challenge-2019-fastai-starter')
from arch import GeM, L2Norm
class GeMNet(nn.Module):
    def __init__(self, new_model):
        super().__init__()
        self.cnn =  new_model.features
        self.head = nn.Sequential(nn.ReLU(),
                                  GeM(3.0),
                                  Flatten(),
                                  L2Norm())
    def forward(self, x):
        x = self.cnn(x)
        out = self.head(x)
        return out
InferenceNet =  GeMNet(models.densenet121(pretrained=True))
OUT_DIM = 1024

In [ ]:
#We will extract database descriptors, then query descriptors and then calculate mean average precision (mAP)
from fastprogress import master_bar, progress_bar
def extract_vectors(data,model):
    model.cuda()
    model.eval()
    with torch.no_grad():
        vectors = torch.zeros(len(data.train_dl), OUT_DIM).cpu()
        idx=0
        for img_label in progress_bar(data.train_dl):
            img,label = img_label
            vectors[idx,:] = model(img).cpu()
            idx+=1
    return vectors
db_vectors = extract_vectors(data,InferenceNet)

In [ ]:
def open_image_cropped(fn:PathOrStr, div:bool=True, convert_mode:str='RGB', cls:type=Image)->Image:
    "Return `Image` object created from image in file `fn`."
    #fn = getattr(fn, 'path', fn)
    x = PIL.Image.open(fn).convert(convert_mode).crop(bbxs[str(fn).replace('./','')])
    x = pil2tensor(x,np.float32)
    if div: x.div_(255)
    return cls(x)

class ImageItemCrop(ImageList):
    def open(self, fn:PathOrStr)->Image:
        return open_image_cropped(fn)
BS=1
NUM_WORKERS=8
tfms = get_transforms(do_flip=False)
tfms = (tfms[1],tfms[1]) #no transforms

query_data = (ImageItemCrop.from_df(qdf,'', cols=['qimages'])
        .split_none()
        .label_const()
        .transform(tfms, resize_method=ResizeMethod.NO)
        .databunch(bs=BS, num_workers=NUM_WORKERS)
        .normalize(imagenet_stats)
       ) 
query_data.train_dl.dl.batch_sampler.sampler = torch.utils.data.SequentialSampler(query_data.train_ds)
query_data.train_dl.dl.batch_sampler.drop_last = False
#No shuffle
query_data.show_batch()

In [ ]:
query_vectors = extract_vectors(query_data,InferenceNet)

In [ ]:
print('>> {}: Evaluating...'.format(dataset))
# convert to numpy
vecs = db_vectors.numpy()
qvecs = query_vectors.numpy()
# search, rank, and print
scores = np.dot(vecs, qvecs.T)
ranks = np.argsort(-scores, axis=0)
compute_map_and_print(dataset, ranks, cfg['gnd'])

this might be bad, smlyaka do 89 something in oxford set and got 33 in retrieval 2019

# Train val good

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="1"
!pip install pretrainedmodels

from fastprogress import master_bar, progress_bar
from fastai.vision import *
from fastai.metrics import accuracy
import pandas as pd
import torch
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch.nn as nn
import pretrainedmodels
import math
import sys
import pickle

In [ ]:
#Lets create validations set
df = pd.read_csv('train_sfm120k_full.csv')

df.Id=df.Id.apply(str)
df.cluster=df.cluster.apply(str)
df.head()

In [ ]:
im_count = df[df.cluster != "-1"].cluster.value_counts()
im_count.name = 'sighting_count'
df = df.join(im_count, on='cluster')
df.head()
path_prefix = 'data/train/retrieval-SfM-120k/ims'
if True:
    val_fns = set(df.sample(frac=1)[(df.cluster != "-1") & (df.sighting_count > 2)].groupby('cluster').first().Image)
    val_fns = list(val_fns)
    val_fns2= []
    for k in val_fns:
        val_fns2.append(path_prefix+'/'+k)
    pd.to_pickle(val_fns2, 'data/val_fns_from_train_full.pkl')

val_fns = pd.read_pickle('data/val_fns_from_train_full.pkl')

Bagian bawah ini keluar error kalo di kaggle, kalo di colab bahkan gak nyampe sini udah ke cancel runtime nya

In [ ]:
SZ = 256
BS = 48
NUM_WORKERS = 10
SEED=0


DATA_DIR='data/'

name = f'DenseNet121-GeMConst-{SZ}-Ring-CELU'
fn2label = {path_prefix+'/'+row[1].Image: row[1].cluster for row in df.iterrows()}
data = (
    ImageList
        .from_df(df[df.cluster != "-1"], path=path_prefix, cols=['Image'])
        .split_by_valid_func(lambda path: path in val_fns)
        .label_from_func(lambda path: fn2label[path])
        .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path='')
        .normalize(imagenet_stats)
)
data.show_batch(rows=3)


In [ ]:
NUM_CLASSES= len(data.classes)
print (NUM_CLASSES)

In [ ]:
from arch import GeM, L2Norm

# Based on my solution to whale-identification-competition
# https://medium.com/@ducha.aiki/thanks-radek-7th-place-solution-to-hwi-2019-competition-738624e4c885
class RingHead(nn.Module):
    def __init__(self, num_classes, feat_dim, in_feat = 1024, r_init =1.5):
        super(RingHead,self).__init__()
        self.eps = 1e-10
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        self.feature_extractor = nn.Sequential(
                        nn.ReLU(),
                        GeM(3.74), Flatten(),
                        nn.BatchNorm1d(in_feat, eps=1e-05, momentum=0.1,
                                                       affine=True, track_running_stats=True),
                        nn.Dropout(p=0.3),
                        nn.Linear(in_features=in_feat, out_features=feat_dim, bias=True),
                        nn.CELU(inplace=True),
                        nn.BatchNorm1d(feat_dim,eps=1e-05, momentum=0.1,
                                                       affine=True, track_running_stats=True))
        
        self.ring =  nn.Parameter(torch.ones(1).cuda()*r_init)
        self.clf = nn.Sequential(nn.Dropout(p=0.5),
                        nn.Linear(in_features=feat_dim, out_features=num_classes, bias=False))
    def forward(self, x):
        feats = self.feature_extractor(x)
        preds = self.clf(feats)
        return preds,feats

class RingGeMNet(nn.Module):
    def __init__(self, new_model, n_classes, in_feats=1024, out_feats=1024):
        super().__init__()
        self.cnn =  new_model.features
        self.head = RingHead(NUM_CLASSES, out_feats, in_feats)
    def forward(self, x):
        x = self.cnn(x)
        preds,feats = self.head(x)
        return preds,feats
@dataclass
class RingLoss(Callback):
    learn:Learner
    alpha:float=0.01
    def on_loss_begin(self, last_output:Tuple[tensor,tensor], **kwargs):
        "Save the extra outputs for later and only returns the true output."
        self.feature_out = last_output[1]
        return {'last_output':last_output[0]}

    def on_backward_begin(self,
                          last_loss:Rank0Tensor,
                          **kwargs):
        
        x = self.feature_out
        R = self.learn.model.head.ring
        loss = None
        cc=0
        x_norm = x.pow(2).sum(dim=1).pow(0.5)
        diff = torch.mean(torch.abs(x_norm - R.expand_as(x_norm))**2)
        if loss is None:
            loss = diff.mean()
        else:
            loss = loss + diff.mean()
        if self.alpha != 0.:  last_loss += (self.alpha * loss).sum()
        return {'last_loss':last_loss}

In [ ]:
learn = Learner(data, RingGeMNet(models.densenet121(pretrained=True), NUM_CLASSES),
                   metrics=[accuracy],
                   loss_func=nn.CrossEntropyLoss(),
                   callback_fns = [RingLoss])
learn.split([learn.model.cnn, learn.model.head])
learn.freeze()
learn.clip_grad();

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(20, 1e-2)

In [ ]:
learn.recorder.plot_losses()
learn.recorder.plot_metrics()

In [ ]:
name = f'DenseNet121-GeM-{SZ}-Ring-CELU'
learn.save(name)

In [ ]:
learn.unfreeze()
max_lr = 1e-3
lrs = [max_lr/5., max_lr]
learn.fit_one_cycle(32, lrs)
learn.recorder.plot_losses()
learn.recorder.plot_metrics()
learn.save(name+"_unfreeze")
#Accuracy gain seems minor, but key factor here is to finetune backbone network

In [ ]:
from fastai import *
from fastai.vision import *
import torch
from fastprogress import master_bar, progress_bar
import PIL
import matplotlib.pyplot as plt
from third_party.testdataset import configdataset
from third_party.evaluate import compute_map_and_print

import pandas as pd


def open_image_cropped(fn:PathOrStr, div:bool=True, convert_mode:str='RGB', cls:type=Image)->Image:
    "Return `Image` object created from image in file `fn`."
    #fn = getattr(fn, 'path', fn)
    x = PIL.Image.open(fn).convert(convert_mode).crop(bbxs[str(fn).replace('./','')])
    x = pil2tensor(x,np.float32)
    if div: x.div_(255)
    return cls(x)

class ImageItemCrop(ImageList):
    def open(self, fn:PathOrStr)->Image:
        return open_image_cropped(fn)

def extract_vectors_batched(data,model,bs):
    model.cuda()
    model.eval()
    num_img = len(data.train_ds)
    vectors = None
    with torch.no_grad():
        idx=0
        for img_label in progress_bar(data.train_dl):
            st=idx*bs
            fin=min((idx+1)*bs, num_img)
            img,label = img_label
            out = model(img).cpu()
            if vectors is None:
                vectors = torch.zeros(num_img, out.size(1))
            vectors[st:fin,:] = out
            idx+=1
    return vectors

def validate_on_dataset(model, dataset_name='oxford5k', DATA_DIR='data/'):
    cfg = configdataset(dataset_name, os.path.join(DATA_DIR, 'test'))
    images = [cfg['im_fname'](cfg,i) for i in range(cfg['n'])]
    qimages = [cfg['qim_fname'](cfg,i) for i in range(cfg['nq'])]
    df = pd.DataFrame(images, columns=['Image'])
    qdf  = pd.DataFrame(qimages, columns=['qimages'])
    global bbxs
    bbxs = {qimages[i]:tuple(cfg['gnd'][i]['bbx']) for i in range(cfg['nq'])}
    BS=1
    NUM_WORKERS=8
    tfms = get_transforms(do_flip=False)
    tfms = (tfms[1],tfms[1]) #no transforms
    query_data = (ImageItemCrop.from_df(qdf,'', cols=['qimages'])
        .split_none()
        .label_const()
        .transform(tfms, resize_method=ResizeMethod.NO)
        .databunch(bs=BS, num_workers=NUM_WORKERS)
        .normalize(imagenet_stats)
       )
    query_data.train_dl.dl.batch_sampler.sampler = torch.utils.data.SequentialSampler(query_data.train_ds)
    query_data.train_dl.dl.batch_sampler.drop_last = False
    print ('Extracting query features...')
    query_vectors = extract_vectors_batched(query_data,model, 1)
    data = (ImageList.from_df(df,'', cols=['Image'])
            .split_none()
            .label_const()
            .transform(tfms, resize_method=ResizeMethod.NO)
            .databunch(bs=BS, num_workers=NUM_WORKERS)
            .normalize(imagenet_stats)
           )
    data.train_dl.dl.batch_sampler.sampler = torch.utils.data.SequentialSampler(data.train_ds)
    data.train_dl.dl.batch_sampler.drop_last = False
    print ('Extracting index features...')
    db_vectors = extract_vectors_batched(data,model,1)
    print('>> {}: Evaluating...'.format(dataset_name))
    # convert to numpy
    vecs = db_vectors.numpy()
    qvecs = query_vectors.numpy()
    # search, rank, and print
    scores = np.dot(vecs, qvecs.T)
    ranks = np.argsort(-scores, axis=0)
    compute_map_and_print(dataset_name, ranks, cfg['gnd'])
    return vecs, qvecs #If you want to check some kind of query expansion

In [ ]:
#Neccessary functions from validation notebook are already in utils.py. So lets just import them
#from utils import validate_on_dataset

#Lets create inference network without clf output and with normalization:
class RingGeMNetInfer(nn.Module):
    def __init__(self, new_model, n_classes, in_feats=1024, out_feats=1024):
        super().__init__()
        self.cnn =  new_model.features
        self.head = RingHead(NUM_CLASSES, out_feats, in_feats)
    def forward(self, x):
        x = self.cnn(x)
        preds,feats = self.head(x)
        return L2Norm()(feats)

InferenceNet =  RingGeMNetInfer(models.densenet121(pretrained=True),
                               NUM_CLASSES)
InferenceNet.load_state_dict(learn.model.state_dict())


val_index_feats, val_query_feats = validate_on_dataset(InferenceNet,'oxford5k')

In [ ]:

#What about original barebone network?
class GeMNet(nn.Module):
    def __init__(self, new_model):
        super().__init__()
        self.cnn =  new_model.features
        self.head = nn.Sequential(nn.ReLU(),
                                  GeM(3.0),
                                  Flatten(),
                                  L2Norm())
    def forward(self, x):
        x = self.cnn(x)
        out = self.head(x)
        return out

InferenceNet =  GeMNet(models.densenet121(pretrained=True))
val_index_feats, val_query_feats = validate_on_dataset( InferenceNet,'oxford5k')

In [ ]:
print ('now finetuned network')
InferenceNet.cnn.load_state_dict(learn.model.cnn.state_dict())
val_index_feats, val_query_feats = validate_on_dataset(InferenceNet, 'oxford5k')

#Nice! We have improved from 41 mAP on Oxford5k to 50 in just 45 minutes.
# It also seems that learned "head" is too specific to our training dataset, so better to discard it

In [ ]:
# Now let`s check and idea from https://arxiv.org/pdf/1902.05509.pdf
# train GeM with p=3 with small images (256px), 
# and test on big (1024px), but with increased p=5

class GeMNet(nn.Module):
    def __init__(self, new_model):
        super().__init__()
        self.cnn =  new_model.features
        self.head = nn.Sequential(nn.ReLU(),
                                  GeM(5.0),
                                  Flatten(),
                                  L2Norm())
    def forward(self, x):
        x = self.cnn(x)
        out = self.head(x)
        return out
    
InferenceNet =  GeMNet(models.densenet121(pretrained=True))
InferenceNet.cnn.load_state_dict(learn.model.cnn.state_dict())
validate_on_dataset( InferenceNet, 'oxford5k')

#Seems to work!


# Submission

In [ ]:
import matplotlib.pyplot as plt
from third_party.testdataset import configdataset
from third_party.evaluate import compute_map_and_print

from fastai import *
from fastai.vision import *
import pandas as pd
from arch import RingGeMNet, GeMNet
from losses import RingLoss
from utils import extract_vectors_batched

In [ ]:
#I suppose, that competition dataset is downloaded to COMP_DATA_DIR
#and the list of images are in index_image_list_fullpath.txt
COMP_DATA_DIR = '/mnt/fry2/users/datasets/landmarkscvprw18/retrieval'
df = pd.read_csv(os.path.join(COMP_DATA_DIR, 'index_image_list_fullpath.txt'),
                  usecols=[0], names=['Image'])
df.head()

In [ ]:

class ImageListAbsPath(ImageList):
    def open(self, fn:PathOrStr)->Image:
        return open_image(fn.replace('./',''))
tfms = get_transforms(do_flip=False)
tfms = (tfms[1],tfms[1]) 
DO_FULL_SIZE = False 
# Extracting features from full-size images would take 15 hours. You probably want to do this 
#for real submission, not for sample. So lets do it on 256x256 images
if DO_FULL_SIZE:
    BS=1
    NUM_WORKERS=8
    data = (ImageListAbsPath.from_df(df,path='', cols=['Image'])
            .split_none()
            .label_const()
            .transform(tfms, resize_method=ResizeMethod.NO)
            .databunch(bs=BS, num_workers=NUM_WORKERS)
            .normalize(imagenet_stats)
           ) 
    data.train_dl.dl.batch_sampler.sampler = torch.utils.data.SequentialSampler(data.train_ds)
    data.train_dl.dl.batch_sampler.drop_last = False
if not DO_FULL_SIZE:
    BS=64
    NUM_WORKERS=8
    data = (ImageListAbsPath.from_df(df,path='', cols=['Image'])
            .split_none()
            .label_const()
            .transform(tfms, resize_method=ResizeMethod.SQUISH, size=192)
            .databunch(bs=BS, num_workers=NUM_WORKERS)
            .normalize(imagenet_stats)
           ) 
    data.train_dl.dl.batch_sampler.sampler = torch.utils.data.SequentialSampler(data.train_ds)
    data.train_dl.dl.batch_sampler.drop_last = False
    #index_features = extract_vectors_batched(data,InferenceNet, BS)
#torch.save(index_features, 'densenet121_pretrained_256px_index_feats.pth')

In [ ]:
#Now we will load training network and transfer its weights to the inference net
TRAIN_CLASSES=780 #that is hardcoded from number of classes in training dataset
learn = Learner(data, RingGeMNet(models.densenet121(pretrained=True), TRAIN_CLASSES),
                   metrics=[accuracy],
                   loss_func=nn.CrossEntropyLoss())

InferenceNet =  GeMNet(models.densenet121())
InferenceNet.cnn.load_state_dict(learn.model.cnn.state_dict())

In [ ]:
#This will take 30 min on Titan X
if not DO_FULL_SIZE:
    qdf = pd.read_csv(os.path.join(COMP_DATA_DIR, 'test_image_list_fullpath.txt'),
                  usecols=[0], names=['Image'])
    qdf.head()
    BS=64
    NUM_WORKERS=6
    qdata = (ImageListAbsPath.from_df(qdf,path='', cols=['Image'])
            .split_none()
            .label_const()
            .transform(tfms, resize_method=ResizeMethod.SQUISH, size=256)
            .databunch(bs=BS, num_workers=NUM_WORKERS)
            .normalize(imagenet_stats)
           ) 
    qdata.train_dl.dl.batch_sampler.sampler = torch.utils.data.SequentialSampler(qdata.train_ds)
    qdata.train_dl.dl.batch_sampler.drop_last = False
    #query_features = extract_vectors_batched(qdata,InferenceNet, BS)
#torch.save(query_features, 'densenet121_pretrained_256px_query_feats.pth')

In [ ]:
query_features = torch.load('densenet121_pretrained_256px_query_feats.pth').numpy()
index_features = np.zeros((len(data.train_ds),1024)).astype(np.float32)

In [ ]:
import faiss

query_fnames = [x.split('/')[-1].replace('.jpg','') for x in qdf.Image.tolist()]
index_fnames = [x.split('/')[-1].replace('.jpg','') for x in df.Image.tolist()]


In [ ]:
from fastprogress import master_bar, progress_bar

def get_idxs_and_dists(query_features, index_features, BS = 32):
    flat_config = faiss.GpuIndexFlatConfig()
    flat_config.device = 0
    res = faiss.StandardGpuResources()
    co = faiss.GpuClonerOptions()
    FEAT_DIM = index_features.shape[1]
    cpu_index = faiss.IndexFlatL2(FEAT_DIM)
    cpu_index.add(index_features)
    index = faiss.index_cpu_to_gpu(res, 0, cpu_index, co)
    out_dists = np.zeros((len(query_features), 100), dtype=np.float32)
    out_idxs = np.zeros((len(query_features), 100), dtype=np.int32)
    NUM_QUERY = len (query_features)
    for ind in progress_bar(range(0, len(query_features), BS)):
        fin = ind+BS
        if fin > NUM_QUERY:
            fin = NUM_QUERY
        q_descs = query_features[ind:fin]
        D, I = index.search(q_descs, 100)
        out_dists[ind:fin] = D
        out_idxs[ind:fin] = I
    return out_idxs, out_dists

def create_submission_from_features(query_features,
                                    index_features,
                                    fname,
                                    query_fnames,
                                    index_fnames):
    out_idxs, out_dists = get_idxs_and_dists(query_features, index_features, BS = 32)
    print (f'Writing {fname}')
    with open(fname, 'w') as f:
        f.write('id,images\n')
        for i in progress_bar(range(len(query_fnames))):
            ids = [index_fnames[x] for x in out_idxs[i]]
            f.write(query_fnames[i] + ',' + ' '.join(ids)+'\n')
    print('Done!')
    return
#create_submission_from_features(query_features, index_features, 'test_submission.csv',query_fnames, index_fnames)